In [27]:
# noqa
from qdrant_client import QdrantClient
import numpy as np


import psycopg
from psycopg.rows import dict_row


## Fictional Warehouses

In [3]:
warehouses = [
    {
        "warehouse_id": "DE-BER-01",
        "warehouse_location": "Berlin, Germany",
        "warehouse_name": "Berlin Distribution Center",
    },
    {
        "warehouse_id": "DE-MUN-01",
        "warehouse_location": "Munich, Germany",
        "warehouse_name": "Munich Logistics Hub",
    },
    {
        "warehouse_id": "DE-HAM-01",
        "warehouse_location": "Hamburg, Germany",
        "warehouse_name": "Hamburg North Warehouse",
    },
    {
        "warehouse_id": "FR-PAR-01",
        "warehouse_location": "Paris, France",
        "warehouse_name": "Paris Central Depot",
    },
    {
        "warehouse_id": "FR-LYO-01",
        "warehouse_location": "Lyon, France",
        "warehouse_name": "Lyon Regional Warehouse",
    },
    {
        "warehouse_id": "FR-MAR-01",
        "warehouse_location": "Marseille, France",
        "warehouse_name": "Marseille Mediterranean Hub",
    },
]

## Simulate Stock Availability for all of the Warehouses

In [4]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [5]:
dummy_vector = np.zeros(1536).tolist()
COLLECTION = "Amazon-items-collection-01-hybrid-search"

payload = qdrant_client.query_points(
    collection_name=COLLECTION,
    query=dummy_vector,
    using="text-embedding-3-small",
    limit=1000,
    with_payload=[
        "parent_asin",
    ],
    with_vectors=False,
)

In [6]:
payload.points

[ScoredPoint(id=416, version=2, score=0.0, payload={'parent_asin': 'B09Q5W9HPQ'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=14, version=2, score=0.0, payload={'parent_asin': 'B0BY2ZBRZD'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=148, version=2, score=0.0, payload={'parent_asin': 'B0CGHVQ4M3'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=360, version=2, score=0.0, payload={'parent_asin': 'B0C78HVVL2'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=417, version=2, score=0.0, payload={'parent_asin': 'B08P6B8GR9'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=457, version=2, score=0.0, payload={'parent_asin': 'B0C64SQ23S'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=138, version=2, score=0.0, payload={'parent_asin': 'B0BYSRTQNQ'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=296, version=2, score=0.0, payload={'parent_asin': 'B0BTKWY6XG'}, vector=No

In [7]:
len([item.payload["parent_asin"] for item in payload.points])

500

In [9]:
product_ids = [item.payload["parent_asin"] for item in payload.points]

# Generate Synthetic availability for items in Qdrant

In [8]:
import random


def generate_inventory_data(warehouses, product_ids, availability_rate=0.75):
    inventory_records = []

    for warehouse in warehouses:
        for product_id in product_ids:
            # 75% chance the product is available in this warehouse
            if random.random() < availability_rate:
                total_quantity = random.randint(0, 100)

                # Only add to inventory if quantity > 0
                if total_quantity > 0:
                    inventory_records.append(
                        {
                            "warehouse_id": warehouse["warehouse_id"],
                            "warehouse_location": warehouse["warehouse_location"],
                            "warehouse_name": warehouse["warehouse_name"],
                            "product_id": product_id,
                            "total_quantity": total_quantity,
                            "reserved_quantity": 0,  # Starting with no reservations
                        }
                    )

    return inventory_records

In [10]:
inventory_data = generate_inventory_data(warehouses, product_ids)

In [ ]:
len(inventory_data), 3000 * 0.75

(2182, 2250.0)

# Insert inventory data to Postgres

In [25]:
def insert_inventory_to_db(inventory_data):
    try:
        conn = psycopg.connect(
            host="localhost",
            port=5433,
            dbname="langgraph_db",
            user="langgraph_user",
            password="langgraph_password",
            autocommit=True,
        )

        with conn.cursor(row_factory=dict_row) as cursor:
            insert_query = """
            INSERT INTO warehouses.inventory 
            (warehouse_id, warehouse_location, warehouse_name, product_id, total_quantity, reserved_quantity)
            VALUES (%(warehouse_id)s, %(warehouse_location)s, %(warehouse_name)s, %(product_id)s, %(total_quantity)s, %(reserved_quantity)s)
            """

            # Use execute_batch for better performance with many inserts
            cursor.executemany(insert_query, inventory_data)

            # Commit the transaction
            conn.commit()

            print(
                f"Successfully inserted {len(inventory_data)} records into warehouses.inventory"
            )

            # Close cursor and connection
            cursor.close()
            conn.close()
    except psycopg.Error as e:
        print(e)
        if conn:
            conn.rollback()
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [26]:
insert_inventory_to_db(inventory_data)

Successfully inserted 2182 records into warehouses.inventory
